In [9]:
import re
from pathlib import Path
import pandas as pd

In [21]:
seeds = [85, 25, 121, 61, 73, 37, 97, 49, 109, 13]

for seed in seeds:
    lgb_file_regex = re.compile(fr"^lgb_.+_seed{seed}\.csv$")
    nn_file_regex = re.compile(fr"^nn_.+_seed{seed}\.csv$")
    lgb_folder = Path("data/predictions/lgb")
    nn_folder = Path("data/predictions/nn")
    
    lgb_file = next((p for p in lgb_folder.iterdir() if lgb_file_regex.match(p.name)), None)
    nn_file = next((p for p in nn_folder.iterdir() if nn_file_regex.match(p.name)), None)

    df_lgb = pd.read_csv(lgb_file)
    df_nn = pd.read_csv(nn_file)

    merged = df_lgb.merge(df_nn, on=["original_song_id", "original_msno"], how="left", suffixes=("_lgb", "_nn"))
    merged['final_pred'] = 0.6 * merged['prediction_lgb'] + 0.4 * merged['prediction_nn'] # using https://www.kaggle.com/competitions/kkbox-music-recommendation-challenge/writeups/bing-bai-a-brief-introduction-to-the-1st-place-sol
    df_final = merged[['original_song_id', 'original_msno', 'final_pred']]
    df_final.columns = ['song_id', 'msno', 'probability']
    df_final.to_csv(f"data/predictions/final/final_seed{seed}.csv", index=False)